<a href="https://colab.research.google.com/github/Dawoon-Jeong0523/Product-Science/blob/main/Product_Lineage_Life_Cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install adjusttext

In [ ]:
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
from numpy import dot
from numpy.linalg import norm
import networkx as nx
from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from networkx.algorithms.community import greedy_modularity_communities
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from adjustText import adjust_text
from math import pi
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import cm
from collections import Counter
import pickle
import os


input_directory="/content/drive/MyDrive/Colab Notebooks/PLLC/Data"
output_directory="/content/drive/MyDrive/Colab Notebooks/PLLC/Output_paper"

# Functions

In [ ]:
def cosine_similarity(A,B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def Ancestor_Descendant_Single(descendant_product,product_list):
  if descendant_product.year==min([product.year for product in product_list]):
    return None

  else:
    temp_list=[]
    for product in product_list:
      if product.year < descendant_product.year:
        temp_value=cosine_similarity(product.genotype,descendant_product.genotype)
        if temp_value>=0.5:
          temp_list.append(temp_value)
          #temp_list.append(Gower_similarity(product.representative_vector,descendant_product.representative_vector,Cat_list))
        else:
          temp_list.append(-1)
      else:
        temp_list.append(-1)
    if max(temp_list)<0:
      return None
    else:
      return product_list[temp_list.index(max(temp_list))]

In [ ]:
class Product():
  def __init__(self,id:int, name:str, year:int, raw_gene:list):
    self.id=id
    self.name=name
    self.year=year
    self.raw_gene=raw_gene
    self.product_type=None
    self.genotype=None
    self.taxon=None
    self.ancestor=None
    self.descendant=None
    self.ancestors=None
    self.descendants=None
    self.information_content=None
    self.innovativeness=None

In [ ]:
def PLLC(temp_product):
  temp_product_lineage=[]
  temp_product_lineage.append(temp_product)
  while True:
    temp_ancestor=temp_product.ancestor
    if temp_ancestor:
      temp_product_lineage.append(temp_ancestor)
      temp_product=temp_ancestor
    else:
      break

  temp_product_lineage.reverse()

  fig=plt.figure(figsize=(20,10))


  key=temp_product_lineage[-1].name
  temp_x_list=[]
  for index in range(2,len(temp_product_lineage)):
    temp_x_list.append(
        np.average(
            [
            temp_product_lineage[index-2].innovativeness,
            temp_product_lineage[index-1].innovativeness,
            temp_product_lineage[index].innovativeness
            ]
        )
    )


  texts=[]
  plt.plot([temp_product_lineage[x].year for x in range(2,len(temp_product_lineage))],temp_x_list,label=key,linewidth=5,alpha=0.5)
  for index in range(2,len(temp_product_lineage)):
    texts.append(plt.text(temp_product_lineage[index].year,temp_x_list[index-2],temp_product_lineage[index].name,fontsize=15))

  plt.xticks([year for year in range(1995,2022)])
  #ax.set_xticklabels([year for year in range(1995,2022)])
  plt.title(f"Product Lineage Life Cycle of %s"%str(temp_product_lineage[-1].name),fontsize=15)
  adjust_text(texts,arrowprops=dict(arrowstyle='->', color='red'))

  plt.xlabel("Year", fontsize=15)
  plt.ylabel("Product Innovativeness",fontsize=15)

  plt.savefig(output_directory+f"/PLLC_%s.jpg"%str(temp_product_lineage[-1].name),dpi=600,transparent=True,bbox_inches="tight")
  return temp_product_lineage

# Data Preprocessing

In [ ]:
gene_dictionary_df=pd.read_csv(input_directory+"/220914_GSMArena_gene_dictionary.csv")
gene_dictionary_df.head()

In [ ]:
df=pd.read_csv(input_directory+"/220914_GSMArena_products_None.csv")
vectors=np.load(input_directory+"/220914_GSMArena_products_None.npy")
df["vector"]=list(vectors)

In [ ]:
df=df[df["year"]!="No"]
df.reset_index(inplace=True,drop=True)
df["year"]=df["year"].astype(float)
df["year"]=df["year"].astype(int)
df["id"]=df["id"].astype(float)
df["id"]=df["id"].astype(int)

In [ ]:
temp_list=[]
for index in range(len(df)):
  temp_list.append([int(x.strip()) for x in df.loc[index,"raw_gene"][1:-1].split(",")])
df["raw_gene"]=temp_list

In [ ]:
product_list=[]
for i in range(len(df)):
  temp_id=i
  temp_name=df.loc[i,"name"]
  temp_year=df.loc[i,"year"]
  # set 부여
  temp_raw_gene=set(df.loc[i,"raw_gene"])
  product_list.append(Product(temp_id,temp_name,temp_year,temp_raw_gene))
  product_list[-1].genotype=df.loc[i,"vector"]

In [ ]:
year_list=[year for year in range(1995,2022)]
product_list=[product for product in product_list if product.year in year_list]

In [ ]:
# Matching Evolutionary Relationship
for product in tqdm(product_list):
  product.ancestor=Ancestor_Descendant_Single(product,product_list)

In [ ]:
gene_dictionary_df["index"]=gene_dictionary_df["index"].astype(int)

In [ ]:
temp_prob_list=[]
for year in year_list:
  temp_products=[product for product in product_list if product.year==year]
  temp_genes=[]
  index=0
  for product in temp_products:
    temp_genes+=product.raw_gene
  temp_dict=Counter(temp_genes)
  temp_sum=sum(Counter(temp_genes).values())
  temp_prob_dict=dict(zip(
      [x for x in temp_dict.keys()],
      [temp_dict[x]/temp_sum for x in temp_dict.keys()]))
  temp_prob_list.append(temp_prob_dict)

In [ ]:
for product in tqdm(product_list):
  temp_prob_dict=temp_prob_list[year_list.index(product.year)]
  product.information_content=np.sum([(-1)*np.log(temp_prob_dict[x]) for x in product.raw_gene])
  product.innovativeness=product.information_content/len(product.raw_gene)

In [ ]:
temp_gene_dictionary_df=gene_dictionary_df[gene_dictionary_df['Domain_2']=='OS']
temp_gene_dictionary_df[temp_gene_dictionary_df['gene']=='no']

# featurephone index = 7043
Pad = ['pad','tab','Amazon Fire',"Tablet",'tablet',"Pad","Tab"]
Watch = ['watch',"Watch"]

Pad = ['pad','tab','amazon fire',"tablet"]


for product in product_list:
  product.product_type='smart'

for product in product_list:
  if 7043 in product.raw_gene:
    product.product_type="feature"

  else:
    for item in Pad:
      if item in product.name.lower():
        if product.name!='Amazon Fire Phone':
          product.product_type='tablet'
    for item in Watch:
      if item in product.name.lower():
        product.product_type='watch'


In [ ]:
gene_dictionary_df[gene_dictionary_df["gene"]=='ios']

In [ ]:
temp_dict=dict(zip([product.name for product in temp_products],
                   [product.innovativeness for product in temp_products]))

sorted(temp_dict.items(),key=lambda item:item[1])[0:10]

In [ ]:
product=[product for product in product_list if product.name=="Apple iPhone"][0]
temp_products=[item for item in product_list if product.year==item.year]
len(temp_products)
temp_list=[]
for item in temp_products:
  temp_list+=item.raw_gene
temp_list=[x[0] for x in Counter(temp_list).most_common() if x[1]==1]
gene_dictionary_df[gene_dictionary_df['index'].isin(temp_list)]
print(len(list(set(product.raw_gene).intersection(set(temp_list))))/len(product.raw_gene))

In [ ]:
len(list(set(product.raw_gene).intersection(set(temp_list))))

In [ ]:
temp_list=list(set(product.raw_gene).intersection(set(temp_list)))
temp_df=gene_dictionary_df[gene_dictionary_df["index"].isin([int(x) for x in temp_list])]
temp_df[temp_df["gene"]!='no']

In [ ]:
product=[product for product in product_list if product.name=="Samsung T439"][0]
temp_products=[item for item in product_list if product.year==item.year]
len(temp_products)
temp_list=[]
for item in temp_products:
  temp_list+=item.raw_gene
temp_list=[x[0] for x in Counter(temp_list).most_common() if x[1]==1]
gene_dictionary_df[gene_dictionary_df['index'].isin(temp_list)]
print(len(list(set(product.raw_gene).intersection(set(temp_list))))/len(product.raw_gene))

In [ ]:
len(list(set(product.raw_gene).intersection(set(temp_list))))

In [ ]:
temp_list=list(set(product.raw_gene).difference(set(temp_list)))
temp_df=gene_dictionary_df[gene_dictionary_df["index"].isin([int(x) for x in temp_list])]
temp_df[temp_df["gene"]!='no']

In [ ]:
temp_df=gene_dictionary_df[gene_dictionary_df["index"].isin([int(x) for x in product.raw_gene])]
temp_df[temp_df["gene"]!='no']

In [ ]:
temp_list=[x[0] for x in sorted(temp_prob_dict.items(), key=lambda item:-item[1])[:50]]
gene_dictionary_df[gene_dictionary_df["index"].isin(temp_list)]

In [ ]:
temp_list=[x[0] for x in sorted(temp_prob_dict.items(), key=lambda item:-item[1])[-50:]]
gene_dictionary_df[gene_dictionary_df["index"].isin(temp_list)]

In [ ]:
temp_list=[]
temp_products=[product for product in product_list if "Samsung" in product.name if "Note" in product.name if product.year==2020]
for product in temp_products:
  temp_list+=product.raw_gene

temp_list=list(set(temp_list))
list(gene_dictionary_df[gene_dictionary_df["index"].isin(temp_list)]["gene"])

In [ ]:
temp_list=[]
temp_products=[product for product in product_list if "Samsung" in product.name if "S" in product.name if product.year==2021]
for product in temp_products:
  temp_list+=product.raw_gene

temp_list=list(set(temp_list))
list(gene_dictionary_df[gene_dictionary_df["index"].isin(temp_list)]["gene"])

## Class Load and Save

In [ ]:
# Save
#for i in tqdm(range(len(product_list))):
#  temp_name="product_"+str(i)
#  with open(f'/content/drive/MyDrive/Colab Notebooks/PLLC/Class/%s'%(temp_name), 'wb') as file:
#    pickle.dump(product_list[i], file)

In [ ]:
# Load
#product_list=[]
#file_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/PLLC/Class/')
#for f in tqdm(file_list):
#  with open(f'/content/drive/MyDrive/Colab Notebooks/PLLC/Class/%s'%(f), 'rb') as file:
#    product = pickle.load(file)
#    product_list.append(product)

In [ ]:
year_list=[year for year in range(1995,2022)]

In [ ]:
len(product_list)

# Product Lineage

In [ ]:
for product in product_list:
  if product.year==2021:
    if product.name=="Apple iPhone 13 Pro Max":
      temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.year==2021:
    if product.name=="Samsung Galaxy Z Flip3 5G":
      temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name=="Nokia 9210i Communicator":
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name=="ZTE F928":
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name=="ZTE F928":
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
temp_products_name=[
    'Apple Watch Series 7 Aluminum', 'Apple iPad Pro 12.9 (2021)','Apple iPhone 13 Pro Max','Samsung Galaxy Z Flip3 5G','Apple iPhone 13 Pro','Nokia 9210i Communicator'
]

In [ ]:
for product in product_list:
  if product.name==temp_products_name[0]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name==temp_products_name[1]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name==temp_products_name[2]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name==temp_products_name[3]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name==temp_products_name[4]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

In [ ]:
for product in product_list:
  if product.name==temp_products_name[5]:
    temp_product=product

temp_list=[]
product=temp_product
temp_list.append(product)
while product.ancestor:
  print(product.ancestor.name, "("+str(product.ancestor.year)+")")
  product=product.ancestor
  temp_list.append(product)

# Product Lineage Life Cycle

In [ ]:
temp_products=[product for product in product_list if "Apple" in product.name if product.year==2021]
for temp_product in temp_products[0:2]:
  PLLC(temp_product)

In [ ]:
temp_products=[product for product in product_list if "Samsung Galaxy Z Flip3 5G" in product.name]

for temp_product in temp_products:
  PLLC(temp_product)

Product Innovativeness

In [ ]:
temp_tablets=[x for x in product_list if x.product_type=='tablet']
temp_smarts=[x for x in product_list if x.product_type=='smart']
temp_watchs=[x for x in product_list if x.product_type=='watch']
temp_features=[x for x in product_list if x.product_type=='feature']

In [ ]:
fig=plt.figure(figsize=(20,10))

temp_products=[product for product in temp_smarts if "Apple" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Apple")

temp_products=[product for product in temp_smarts if "Samsung" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Samsung")

temp_products=[product for product in temp_smarts if "Nokia" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Nokia")

temp_products=[product for product in temp_smarts if "LG" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="LG")

temp_products=[product for product in temp_smarts if "Motorola" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Motorola")

temp_products=[product for product in temp_smarts if "Xiaomi" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Xiaomi")

plt.xticks(year_list)

plt.xlabel("Year", fontsize=15)
plt.ylabel("Average Product Innovativeness",fontsize=15)
plt.legend(loc="upper left",fontsize=15)

plt.savefig(output_directory+"/Smartphone_firm.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
fig=plt.figure(figsize=(20,10))

temp_products=[product for product in temp_features if "Samsung" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Samsung")

temp_products=[product for product in temp_features if "Nokia" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Nokia")

temp_products=[product for product in temp_features if "LG" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="LG")

temp_products=[product for product in temp_features if "Motorola" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Motorola")

plt.xticks(year_list)

plt.xlabel("Year", fontsize=15)
plt.ylabel("Average Product Innovativeness",fontsize=15)
plt.legend(loc="upper left",fontsize=15)

plt.savefig(output_directory+"/Featurephone_firm.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
fig=plt.figure(figsize=(20,10))

temp_products=[product for product in temp_watchs if "Apple" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Apple")

temp_products=[product for product in temp_watchs if "Samsung" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Samsung")

temp_products=[product for product in temp_watchs if "Nokia" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Nokia")

temp_products=[product for product in temp_watchs if "LG" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="LG")

temp_products=[product for product in temp_watchs if "Motorola" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Motorola")

temp_products=[product for product in temp_watchs if "Xiaomi" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Xiaomi")

plt.xticks(year_list)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Average Product Innovative Degree",fontsize=15)
plt.legend(loc="upper left",fontsize=15)

plt.savefig(output_directory+"/Watch_firm.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
fig=plt.figure(figsize=(20,10))

temp_products=[product for product in temp_tablets if "Apple" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Apple")

temp_products=[product for product in temp_tablets if "Samsung" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Samsung")

temp_products=[product for product in temp_tablets if "LG" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="LG")

temp_products=[product for product in temp_tablets if "Amazon" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Amazon")

temp_products=[product for product in temp_tablets if "Xiaomi" in product.name]
temp_year_list=sorted(list(set([product.year for product in temp_products])))
temp_y_list=[]
for year in temp_year_list:
  temp=[product.innovativeness for product in temp_products if product.year==year]
  temp_y_list.append(np.average(temp))
plt.plot(temp_year_list,temp_y_list,label="Xiaomi")

plt.xticks(year_list)
plt.xlabel("Year", fontsize=15)
plt.ylabel("Average Product Innovativeness",fontsize=15)
plt.legend(loc="upper left",fontsize=15)


plt.savefig(output_directory+"/Tablet_firm.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
for year in year_list:
  temp_dict=dict(zip(
      [product.name for product in product_list if product.year==year],
      [product.innovativeness for product in product_list if product.year==year]
  ))
  temp_str=""
  temp_list=sorted(temp_dict.items(), key=lambda item:-item[1])[0:5]
  for i in range(len(temp_list)):
    x=temp_list[i]
    temp_str+=f'%s (%s)'%(str(x[0]),str(round(x[1],2)))
    temp_str+=', '

  print(year,temp_str)

# Phylogenetic Tree

In [ ]:
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

temp_G=nx.DiGraph()

temp_products=[product for product in product_list if "Apple" in product.name]
for product in temp_products:
  while True:
    if product.ancestor:
      temp_G.add_edge(product.ancestor.name,product.name)
      product=product.ancestor
    else:
      temp_G.add_edge('Root',product.name)
      break

pos1=graphviz_layout(temp_G, prog="dot",root='Root')
temp_pos=pos1
pos1={}
for key in list(temp_pos.keys()):
  pos1[key]=[0,0]
  pos1[key][0]=(temp_pos[key][0])
  pos1[key][1]=(-temp_pos[key][1])


In [ ]:
from sklearn.preprocessing import minmax_scale

temp_scaled=minmax_scale([product.innovativeness for product in product_list], axis=0, copy=True)

i=0
for product in product_list:
  product.ancestors=temp_scaled[i]
  i+=1

In [ ]:
import re

fig=plt.figure(figsize=(20,20))
ax=plt.subplot(111)

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)

for n in temp_G.nodes():
  if "iPhone" in n:
    color="red"
    label='iPhone'
  elif "iPad" in n:
    color="blue"
    label='iPad'
  elif 'Apple Watch' in n:
    color="green"
    label='Apple Watch'
  else:
    color="black"
    label='Others'

  if n=="Root":
    ax.text(pos1[n][0],pos1[n][1],re.sub('Apple ','',n),fontsize=20,rotation=20,color=color,fontweight='bold')

  else:
    ax.text(pos1[n][0],pos1[n][1],re.sub('Apple ','',n),fontsize=10,rotation=20,color=color)



arr1 = ax.arrow(0,0, 0,0, head_width=0, color='red', length_includes_head=True,label="iPhone")
arr2 = ax.arrow(0,0, 0,0, head_width=0, color='blue', length_includes_head=True,label="iPad")
arr3 = ax.arrow(0,0, 0,0, head_width=0, color='green', length_includes_head=True,label="Apple Watch")
arr4 = ax.arrow(0,0, 0,0, head_width=0, color='black', length_includes_head=True,label="Others")

ax.legend([arr1,arr2,arr3,arr4],["iPhone","iPad","Apple Watch","Others"],loc="lower left",fontsize=20)

ax.spines["top"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)

plt.savefig(output_directory+"/Apple Tree.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
temp_G=nx.DiGraph()

temp_products=[product for product in product_list if "Apple" in product.name]
temp_products+=[product for product in product_list if "Sewon" in product.name]
temp_products+=[product for product in product_list if 'Kyocera' in product.name]

for product in temp_products:
  while True:
    if product.ancestor:
      temp_G.add_edge(product.ancestor.name,product.name)
      product=product.ancestor
    else:
      temp_G.add_edge('Root',product.name)
      break

pos1=graphviz_layout(temp_G, prog="dot",root='Root')
temp_pos=pos1
pos1={}
for key in list(temp_pos.keys()):
  pos1[key]=[0,0]
  pos1[key][0]=(temp_pos[key][0])
  pos1[key][1]=(-temp_pos[key][1])


fig=plt.figure(figsize=(20,20))
ax=plt.subplot(111)


for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)


for n in temp_G.nodes():
  if "Apple" in n:
    color="red"
  elif "Sewon" in n:
    color="blue"
  elif 'Kyocera' in n:
    color="green"
  else:
    color="black"

  if n=="Root":
    ax.text(pos1[n][0],pos1[n][1],n.split(' ')[0],fontsize=20,rotation=20,color=color,fontweight='bold')

  else:
    ax.text(pos1[n][0],pos1[n][1],n.split(' ')[0],fontsize=10,rotation=20,color=color)


arr1 = ax.arrow(0,0, 0,0, head_width=0, color='red', length_includes_head=True,label="Apple")
arr2 = ax.arrow(0,0, 0,0, head_width=0, color='blue', length_includes_head=True,label="Sewon")
arr3 = ax.arrow(0,0, 0,0, head_width=0, color='green', length_includes_head=True,label="Kyocera")
arr4 = ax.arrow(0,0, 0,0, head_width=0, color='black', length_includes_head=True,label="Others")

ax.legend([arr1,arr2,arr3,arr4],["Apple","Sewon","Kyocera","Others"],loc="lower left",fontsize=20)

ax.spines["top"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)

plt.savefig(output_directory+"/Apple Sewon Kyocera Tree_color.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
temp_G=nx.DiGraph()

temp_products_name=[
    'Apple Watch Series 7 Aluminum', 'Apple iPad Pro 12.9 (2021)','Apple iPhone 13 Pro Max','Samsung Galaxy Z Flip3 5G','Apple iPhone 13 Pro','Nokia 9210i Communicator'
]

temp_products=[product for product in product_list if product.name in temp_products_name]
#temp_products+=[product for product in product_list if "Nokia" in product.name]

for product in temp_products:
  while True:
    if product.ancestor:
      temp_G.add_edge(product.ancestor.name,product.name)
      product=product.ancestor
    else:
      temp_G.add_edge('Root',product.name)
      break


pos1=graphviz_layout(temp_G, prog="dot",root='Root')
temp_pos=pos1
pos1={}
for key in list(temp_pos.keys()):
  pos1[key]=[0,0]
  pos1[key][0]=(temp_pos[key][0])
  pos1[key][1]=(-temp_pos[key][1])


fig, ax=plt.subplots(figsize=(20,20))

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            #alpha=temp_product.ancestors,
                            alpha=1,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            #alpha=temp_product.ancestors,
                            alpha=1,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)

for n in temp_G.nodes():
  if n=="Root":
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=30,rotation=20,fontweight="bold")
  else:
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=15,rotation=20)

ax.spines["top"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)


# Zoom in 효과- Apple
axins=ax.inset_axes([-0.4, 0.3, 0.3, 0.9])
for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins)

x1, x2, y1, y2 = 100 , 400, -1200, -200
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels([])
axins.set_yticklabels([])

for temp_ax in ['top','bottom','left','right']:
  axins.spines[temp_ax].set_color('red')
  axins.spines[temp_ax].set_linewidth(5)
  axins.spines[temp_ax].set_linestyle('dotted')

for n in temp_G.nodes():
  if x1<=pos1[n][0]<=x2:
    if y1<=pos1[n][1]<=y2:
      axins.text(pos1[n][0],pos1[n][1],n,fontsize=25,rotation=20)
ax.indicate_inset_zoom(axins, edgecolor="red",linewidth=5,linestyle="dotted")


# Zoom in 효과 - Nokia
axins2=ax.inset_axes([-0.2, -0.1, 0.3, 0.3])
for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins2)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins2)

x1, x2, y1, y2 = 800 , 900, -1700, -1500
axins2.set_xlim(x1, x2)
axins2.set_ylim(y1, y2)
axins2.set_xticklabels([])
axins2.set_yticklabels([])

for temp_ax in ['top','bottom','left','right']:
  axins2.spines[temp_ax].set_color('green')
  axins2.spines[temp_ax].set_linewidth(5)
  axins2.spines[temp_ax].set_linestyle('dotted')


for n in temp_G.nodes():
  if x1<=pos1[n][0]<=x2:
    if y1<=pos1[n][1]<=y2:
      axins2.text(pos1[n][0],pos1[n][1],n,fontsize=25,rotation=20)
ax.indicate_inset_zoom(axins2, edgecolor="green",linewidth=5,linestyle="dotted")

# Zoom in 효과- Samsung
axins3=ax.inset_axes([1.2, 0.3, 0.2, 0.8])
for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins3)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0,
                            ax=axins3)

for temp_ax in ['top','bottom','left','right']:
  axins3.spines[temp_ax].set_color('blue')
  axins3.spines[temp_ax].set_linewidth(5)
  axins3.spines[temp_ax].set_linestyle('dotted')

x1, x2, y1, y2 = 1050 , 1100, -800, 0
axins3.set_xlim(x1, x2)
axins3.set_ylim(y1, y2)
axins3.set_xticklabels([])
axins3.set_yticklabels([])

for n in temp_G.nodes():
  if x1<=pos1[n][0]<=x2:
    if y1<=pos1[n][1]<=y2:
      axins3.text(pos1[n][0],pos1[n][1],n,fontsize=25,rotation=20)

ax.indicate_inset_zoom(axins3, edgecolor="blue",linewidth=5,linestyle="dotted")

plt.savefig(output_directory+"/Lineage Sample_Zoom.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
len(product_list)

In [ ]:
temp_firms=[product.name.split(' ')[0] for product in product_list]
firms=[firm[0] for firm in Counter(temp_firms).most_common()[0:9]]+["Apple"]

In [ ]:
temp_products=[product for product in product_list if product.name!='Samsung :) Smiley' if product.name.split(' ')[0] in firms]
len(temp_products)

In [ ]:
temp_G=nx.DiGraph()

for product in temp_products:
  while True:
    if product.ancestor:
      temp_G.add_edge(product.ancestor.name,product.name)
      product=product.ancestor
    else:
      temp_G.add_edge('Root',product.name)
      break

pos1=graphviz_layout(temp_G, prog="dot",root='Root')
temp_pos=pos1
pos1={}
for key in list(temp_pos.keys()):
  pos1[key]=[0,0]
  pos1[key][0]=(temp_pos[key][0])
  pos1[key][1]=(-temp_pos[key][1])


fig=plt.figure(figsize=(300,100))
ax=plt.subplot(111)

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)


#plt.savefig(output_directory+"/All Tree.jpg",dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
fig=plt.figure(figsize=(300,100))
ax=plt.subplot(111)

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)




In [ ]:
fig=plt.figure(figsize=(300,100))
ax=plt.subplot(111)

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)



for n in temp_G.nodes():
  if n=="Root":
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=20,rotation=20,color=color,fontweight='bold')

  else:
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=10,rotation=20,color=color)

In [ ]:
fig=plt.figure(figsize=(600,100))
ax=plt.subplot(111)

for product_name in temp_G.nodes():
  temp_product=[product for product in product_list if product.name==product_name]
  if temp_product:
    temp_product=temp_product[0]
    if temp_product.ancestor:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,temp_product.ancestor.name)],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)
    else:
      nx.draw_networkx_edges(temp_G,
                            pos=pos1,
                            alpha=1,
                             #temp_product.ancestors,
                            edgelist=[(temp_product.name,"Root")],
                            width=5,
                            arrows=False,
                            connectionstyle="arc3,rad=1",
                            edge_color="red",
                            node_size=0)



for n in temp_G.nodes():
  if n=="Root":
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=20,rotation=20,color=color,fontweight='bold')

  else:
    ax.text(pos1[n][0],pos1[n][1],n,fontsize=10,rotation=20,color=color)

# Robustness

In [ ]:
temp_prob_list=[]
for year in year_list:
  temp_products=[product for product in product_list if product.year==year]
  temp_genes=[]
  threshold=5
  for product in temp_products:
    temp_genes+=product.raw_gene
  temp_gene_dict=Counter(temp_genes)
  temp_dict={n:temp_gene_dict[n] for n in temp_gene_dict.keys() if temp_gene_dict[n]>threshold}
  temp_sum=sum(temp_dict.values())
  temp_prob_dict=dict(zip(
      [x for x in temp_dict.keys()],
      [temp_dict[x]/temp_sum for x in temp_dict.keys()]))

  temp_prob_list.append(temp_prob_dict)

  for product in temp_products:
    temp_prob=[temp_prob_dict[x] for x in product.raw_gene if x in temp_dict.keys()]
    product.innovativeness=(-np.sum(np.log(temp_prob)))/len(temp_prob)

temp_entropy_list=[]
for item in temp_prob_list:
  temp_entropy_list.append(np.sum([(-1)*item[x]*np.log(item[x]) for x in item.keys()]))

In [ ]:
fig=plt.figure(figsize=(20,10))
for threshold in [0,1,2,5]:
  temp_prob_list=[]
  for year in year_list:
    temp_products=[product for product in product_list if product.year==year]
    temp_genes=[]
    for product in temp_products:
      temp_genes+=product.raw_gene
    temp_gene_dict=Counter(temp_genes)
    temp_dict={n:temp_gene_dict[n] for n in temp_gene_dict.keys() if temp_gene_dict[n]>threshold}
    temp_sum=sum(temp_dict.values())
    temp_prob_dict=dict(zip(
        [x for x in temp_dict.keys()],
        [temp_dict[x]/temp_sum for x in temp_dict.keys()]))

    temp_prob_list.append(temp_prob_dict)

    for product in temp_products:
      temp_prob=[temp_prob_dict[x] for x in product.raw_gene if x in temp_dict.keys()]
      product.average_information=(-np.sum(np.log(temp_prob)))/len(temp_prob)


  temp_product=[product for product in product_list if product.name=="Apple iPhone 13 Pro Max"][0]
  temp_product_lineage=[]
  temp_product_lineage.append(temp_product)
  while True:
    temp_ancestor=temp_product.ancestor
    if temp_ancestor:
      temp_product_lineage.append(temp_ancestor)
      temp_product=temp_ancestor
    else:
      break

  temp_product_lineage.reverse()
  key=temp_product_lineage[-1].name
  temp_x_list=[]
  for index in range(2,len(temp_product_lineage)):
    temp_x_list.append(
        np.average(
            [
            temp_product_lineage[index-2].average_information,
            temp_product_lineage[index-1].average_information,
            temp_product_lineage[index].average_information
            ]
        )
    )


  texts=[]
  plt.plot([temp_product_lineage[x].year for x in range(2,len(temp_product_lineage))],temp_x_list,linewidth=5,alpha=0.5,label="Frequency threshold_%s"%str(threshold))

plt.xticks([year for year in range(1995,2022)])
plt.legend(fontsize=15,loc="upper left")
plt.title(f"Product Lineage Life Cycle of %s"%str(temp_product_lineage[-1].name),fontsize=15)

adjust_text(texts,arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel("Year", fontsize=15)
plt.ylabel("Product Innovativeness",fontsize=15)
plt.savefig(output_directory+f"/PLLC_robustness_%s.jpg"%str(temp_product_lineage[-1].name),dpi=600,transparent=True,bbox_inches="tight")

In [ ]:
fig=plt.figure(figsize=(20,10))
for threshold in [0,1,2,5]:
  temp_prob_list=[]
  for year in year_list:
    temp_products=[product for product in product_list if product.year==year]
    temp_genes=[]
    for product in temp_products:
      temp_genes+=product.raw_gene
    temp_gene_dict=Counter(temp_genes)
    temp_dict={n:temp_gene_dict[n] for n in temp_gene_dict.keys() if temp_gene_dict[n]>threshold}
    temp_sum=sum(temp_dict.values())
    temp_prob_dict=dict(zip(
        [x for x in temp_dict.keys()],
        [temp_dict[x]/temp_sum for x in temp_dict.keys()]))

    temp_prob_list.append(temp_prob_dict)

    for product in temp_products:
      temp_prob=[temp_prob_dict[x] for x in product.raw_gene if x in temp_dict.keys()]
      product.average_information=(-np.sum(np.log(temp_prob)))/len(temp_prob)


  temp_product=[product for product in product_list if product.name=="Samsung Galaxy Z Flip3 5G"][0]
  temp_product_lineage=[]
  temp_product_lineage.append(temp_product)
  while True:
    temp_ancestor=temp_product.ancestor
    if temp_ancestor:
      temp_product_lineage.append(temp_ancestor)
      temp_product=temp_ancestor
    else:
      break

  temp_product_lineage.reverse()
  key=temp_product_lineage[-1].name
  temp_x_list=[]
  for index in range(2,len(temp_product_lineage)):
    temp_x_list.append(
        np.average(
            [
            temp_product_lineage[index-2].average_information,
            temp_product_lineage[index-1].average_information,
            temp_product_lineage[index].average_information
            ]
        )
    )


  texts=[]
  plt.plot([temp_product_lineage[x].year for x in range(2,len(temp_product_lineage))],temp_x_list,linewidth=5,alpha=0.5,label="Frequency threshold_%s"%str(threshold))

plt.xticks([year for year in range(1995,2022)])
plt.legend(fontsize=15,loc="upper left")
plt.title(f"Product Lineage Life Cycle of %s"%str(temp_product_lineage[-1].name),fontsize=15)

adjust_text(texts,arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel("Year", fontsize=15)
plt.ylabel("Product Innovativeness",fontsize=15)
plt.savefig(output_directory+f"/PLLC_robustness_%s.jpg"%str(temp_product_lineage[-1].name),dpi=600,transparent=True,bbox_inches="tight")